# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Как видим, пропущенных данных нет.
Тип данных столбца messages - float64, что кажется избыточным, так как количество сообщений - число целое. Заменим здесь тип данных.

In [5]:
df['messages'] = df['messages'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 125.7 KB


Проверим наличие явных дубликатов

In [6]:
df.duplicated().sum()

0

Дубликатов не обнаружено

Вывод:
В данных не обнаружено пропущенных значений, дубликатов. Названия столбцов записаны корректно. Столбец с сообщениями приведён к целочисленному типу.

## Разобьём данные на выборки

Поскольку у нас нет спрятанной тестовой выборки, то данные нужно разбить на три части: обучающую, валидационную и тестовую.
Разобъем исходные данные в соотношении 3:1:1, сделав тестовый и валидационный наборы одинакового размера.


In [7]:
df_train, df_test_valid = train_test_split(df, test_size = 0.4, random_state = 12345)

Разобъём наши данные на целевые и признаки

In [8]:
features_train = df_train.drop('is_ultra', axis = 1)
target_train = df_train['is_ultra']

In [9]:
features_test_valid = df_test_valid.drop('is_ultra', axis = 1)
target_test_valid = df_test_valid['is_ultra']

In [10]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test_valid, target_test_valid, test_size = 0.5, random_state = 12345)

Выводы:мы разбили данные на обучающий(60%), тестовый(20%) и валидационный(20%) наборы.

## Исследуем модели

Исследуем модели.Используем дерево решений, случайный лес, логистическую регрессию.

Дерево решений

In [11]:
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model_tree.fit(features_train, target_train)
    result = model_tree.score(features_valid, target_valid)
    if result > best_result:
        best_result = result
        best_depth = depth
print("Лучший результат дерева:", best_result)
print("Лучшая глубина дерева:", best_depth)

Лучший результат дерева: 0.7853810264385692
Лучшая глубина дерева: 3


Случайный лес

In [12]:
best_model_forest = None
best_result_forest = 0
best_depth = 0
for depth in range(1, 11):
    for est in range(3,33,3):
        model = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth = depth)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result_forest:
            best_model_forest = model
            best_result_forest = result
            best_depth = depth
print("Лучший результат леса", best_result_forest)
print("Лучшая глубина леса", best_depth)
print("Лучшая модель: ", best_model_forest)

Лучший результат леса 0.8040435458786936
Лучшая глубина леса 6
Лучшая модель:  RandomForestClassifier(max_depth=6, n_estimators=12, random_state=12345)


Логистическая регрессия

In [13]:
model_regression = LogisticRegression(random_state = 12345, solver = 'lbfgs', max_iter = 1000)

In [14]:
model_regression.fit(features_train, target_train)
accuracy_regression = model_regression.score(features_valid, target_valid)
print("Accuracy при логистической регрессии:", accuracy_regression)

Accuracy при логистической регрессии: 0.7107309486780715


После исследования модели результаты:
дерево решений - 0,785  
случайный лес - 0,804  
логистическая регрессия - 0,711.    
Пока самый высокий результат показал случайный лес, глубиной 6 и количеством деревьев 12.

## Проверим модель на тестовой выборке

Дерево решений

In [15]:
model_tree = DecisionTreeClassifier(random_state = 12345, max_depth = 3)
model_tree.fit(features_train, target_train)
result_tree = model_tree.score(features_test, target_test)
print("Результат дерева решений:", result_tree)

Результат дерева решений: 0.7791601866251944


Случайный лес

In [16]:
model_random_forest = RandomForestClassifier(max_depth = 6, n_estimators = 12,random_state = 12345)
model_random_forest.fit(features_train, target_train)
result_random_forest = model_random_forest.score(features_test, target_test)
print("Результат случайного леса:", result_random_forest)

Результат случайного леса: 0.7947122861586314


Логистическая регрессия

In [17]:
model_logistic_regression = LogisticRegression(solver = 'lbfgs', random_state = 12345, max_iter = 1000)
model_logistic_regression.fit(features_train, target_train)
result_logistic_regression = model_logistic_regression.score(features_test, target_test)
print("Результат логистической регрессии:", result_logistic_regression)

Результат логистической регрессии: 0.6842923794712286


Проверка на тестовой выборке дала значения, близкие к валидационным. Всё так же лидирует случайный лес.

Вывод:   
Исходные данные были изучены, откорректированы и разделены на обучающую, валидационную и тестовую выборки. Исследовались модели. Использовалось дерево решений, случайный лес, логистическая регрессия. Результаты исследования:    
дерево решений - 0,785    
случайный лес - 0,804    
логистическая регрессия - 0,711.    
Самый высокий результат показал случайный лес, глубиной 6 и количеством деревьев 12.   
Модели были проверены на тестовой выборке. Результаты проверки:   
дерево решений - 0,779   
случайный лес - 0,795    
логистическая регрессия - 0,684.        
И снова самый высокий результат у случайного леса.
Была достигнута accuracy выше 0,75